In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
seed = 2
from xgboost import XGBClassifier
import xgboost as xgb
import pickle

In [2]:
data =np.load('./data/ready_data.npy')
targets = pd.read_csv('./data/target')

In [8]:
train = data[:targets.shape[0]]
test = data[targets.shape[0]:]
target = targets.values
print('Train: ', train.shape, '\tTarget: ', target.shape)
print('Test:', test.shape)

Train:  (595211, 369) 	Target:  (595211, 1)
Test: (892817, 369)


In [4]:
# taking sample with 50% of target = 1 and 50% of target =1.

sample_size = int(1e4)
sample = train[train.target == 1].sample(n=int(sample_size/2), random_state=seed)
sample = sample.append( train[train.target == 0].sample(n=int(sample_size/2), random_state=seed))
sample = sample.sample(frac=1, random_state=seed).reset_index(drop=True)
#sample.to_csv('./data/train_sample.csv', index=False)

In [6]:
dtrain = xgb.DMatrix(data, targets, nthread=-1)

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=.2, random_state=seed)

In [11]:
model = XGBClassifier(
	objective = 'binary:logistic',
	n_jobs = -1,
	random_state = seed,
	eval_metric = "auc",
	#-------
    max_delta_step =0
	scale_pos_weight = 1,
	#-----------
    max_depth = 6,
	min_child_weight = 1,
	#----------
	gamma = 0,
	#----------
	subsample = 1,
	colsample_bytree = 1,
	#------------
	reg_alpha = 0,
	reg_lambda = 1,
	#------------
	learning_rate = 0.3

	#n_estimators = 1000,
	)

In [ ]:
with open('./data/parameters.pkl','rb') as f:
    p = pickle.load(f)

In [12]:
model = XGBClassifier()

In [13]:
model.fit(x_train, y_train)

/home/beka/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/beka/.local/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [ ]:
with open('./data/model.pkl', 'wb') as f:
    pickle.dump(file=f, obj=model)

In [ ]:
with open('./data/model.pkl', 'rb') as f:
    model = pickle.load(f)

In [16]:
pred = model.predict(x_test)
print (classification_report(y_true=y_test, y_pred=pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98    114622
           1       0.00      0.00      0.00      4421

    accuracy                           0.96    119043
   macro avg       0.48      0.50      0.49    119043
weighted avg       0.93      0.96      0.94    119043



/home/beka/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
from sklearn.metrics import classification_report


In [17]:
pred = model.predict(test)
sum = pd.Dataframe({'id':range(test.shape[0]), 'target':pred})

AttributeError: module 'pandas' has no attribute 'Dataframe'

In [21]:
sum.target.value_counts()

0    892816
1         1
Name: target, dtype: int64